## Imports

In [ ]:
import csv
import os
import sys
import numpy as np
import warnings

pkg_path = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
sys.path.append(pkg_path)

np.set_printoptions(threshold=np.inf, suppress=True, linewidth=np.inf)
warnings.filterwarnings(action='ignore', category=UserWarning)

import shtmbss2.addsrc
from shtmbss2.common.config import *
from shtmbss2.core.data import load_yaml
from shtmbss2.core.helpers import Latex

## Config

In [ ]:
evaluation_type = 'onchip'
extra_key = "num-epochs"

col_order = [0, -2, -1, 1, 2, 3]

In [ ]:
config_file_name = "shtm2bss_config_evaluation.yaml"
config = load_yaml(PATH_CONFIG, config_file_name)[evaluation_type]

exp_ids = config["exp_ids"]
backends = config["backends"]
target_keys = config["target_keys"]

## Convert results to Latex table

In [ ]:
output = list()
output.append(target_keys + [f"{extra_key}_max"])

# print header row
Latex.print_row(output[-1], col_order)

for i_exp, exp_id in enumerate(exp_ids):
    backend = backends[i_exp]
    folder_path = join(EXPERIMENT_FOLDERS[backend], EXPERIMENT_SUBFOLDERS[ExperimentType.EVAL_MULTI])
    file_path = join(folder_path, f"SHTMTotal_{exp_id}", "performance.csv")
    with open(file_path, 'r') as csvfile:
        csv_reader = csv.reader(csvfile)
        lines = list(csv_reader)
        data = dict(zip(lines[0], lines[1]))
        output_tmp = [data.get(k, "0") for k in target_keys]
    file_path = join(folder_path, f"SHTMTotal_{exp_id}", "experimental_results.csv")
    with open(file_path, 'r') as csvfile:
        csv_reader = csv.reader(csvfile)
        lines = list(csv_reader)
        extra_key_id = lines[0].index(extra_key)
        max_value = -1
        for line in lines[1:]:
            if int(line[extra_key_id]) < 0:
                max_value = "100+"
                break
            max_value = max(int(line[extra_key_id]), max_value)
        min_value = 101
        for line in lines[1:]:
            if int(line[extra_key_id]) > 0:
                min_value = min(int(line[extra_key_id]), min_value)
        if min_value > 100:
            min_value = "100+"
        output_tmp.append(str(min_value))
        output_tmp.append(str(max_value))
    output.append(output_tmp)

    Latex.print_row(output[-1], col_order, exp_id)
